In [1]:
import yaml
from pytorch_lightning.loggers import NeptuneLogger
from Modules.train import DataModule, Model, get_trainer

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


# Training

In [2]:
# %%script false --no-raise-error

with open("Modules/config.yaml", 'r') as stream:
    PARAMS = yaml.safe_load(stream)
    PARAMS = PARAMS['classifier']
    print(PARAMS)

neptune_logger = NeptuneLogger(
        project="kaori/AISeed",
        # with_id="AIS-113",
        log_model_checkpoints=False,
    )

{'task': 'classification', 'architect_settings': {'name': 'classifier-test', 'backbone': {'name': 'resnet-s', 'is_full': False, 'is_pretrained': True, 'is_freeze': False}, 'n_cls': 10}, 'dataset_settings': {'name': 'CIFAR10', 'path': 'data/', 'img_size': 224}, 'training_settings': {'gpu_ids': [0], 'n_gpu': 1, 'loss': 'ce', 'ckpt_path': './mode/test', 'n_epoch': 5, 'n_batch': 64, 'num_workers': 0, 'optimizer': 'adam', 'lr_scheduler': 'step', 'early_stopping': True, 'lr': 0.0001, 'lr_step': 10, 'lr_decay': 0.8, 'momentum': 0.9, 'weight_decay': 0.005}}


In [3]:
# %%script false --no-raise-error

import albumentations as A
from albumentations.pytorch import ToTensorV2

# img_size = PARAMS['dataset_settings']['img_size']
# transforms_img = A.Compose([
#     A.RandomResizedCrop(img_size, img_size),  # Random crop and resize
#     A.HorizontalFlip(),  # Random horizontal flip
#     A.VerticalFlip(),  # Random vertical flip
#     A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     ToTensorV2(),  # Convert to tensor
# ])

data = DataModule(PARAMS['dataset_settings'], PARAMS['training_settings'], [None, None])

In [4]:
# %%script false --no-raise-error
neptune_logger.log_hyperparams(params=PARAMS)
model = Model(PARAMS=PARAMS)
trainer = get_trainer(PARAMS['training_settings'], neptune_logger)
# train
trainer.fit(model, data)
# test
trainer.test(model, data)

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  warnings.warn(


https://app.neptune.ai/kaori/AISeed/e/AIS-119


c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'list'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | WrappingClassifier | 11.2 M
1 | train_metrics | MulticlassAccuracy | 0     
2 | valid_metrics | MulticlassAccuracy | 0     
3 | loss          | CrossEntropyLoss   | 0     
-----------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


...

Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test metric               DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
metrics/epoch/test_accuracy    0.9093000292778015
  metrics/epoch/test_loss      0.28274163603782654
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'metrics/epoch/test_loss': 0.28274163603782654,
  'metrics/epoch/test_accuracy': 0.9093000292778015}]

In [5]:
# model = Model.load_from_checkpoint(".neptune/Untitled/AIS-113/checkpoints/best_model_001.ckpt")
# trainer = get_trainer(PARAMS['training_settings'], neptune_logger)
# # 
# trainer.test(model, data)

# Testing

In [6]:
%%script false --no-raise-error
import torch
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import os
from torchvision import transforms

with open("models/class_name.txt", "r", encoding='utf-8') as f:
    class_names = f.read().splitlines()

PARAMS =  {
    "architect_settings" : {
            "name": "model-test",
            "backbone": {
                    "name": "maskrcnn-s",
                    "is_full": False,
                    "is_pretrained": True,
                    "is_freeze": False, 
            },
            "n_cls": 2,
            },
    "dataset_settings": {
            
            },
    "training_settings":{
    
    }
}

model_list = [name.split('.')[0] for name in os.listdir("models") if name.endswith('ckpt')]
model_list += ["fcn", "resnet", "fasterrcnn", "maskrcnn"]
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)

def predict(image, model_choice):
    labels, detection = None, None
    PARAMS['architect_settings']['backbone']['is_full'] = True
    if(model_choice == "fcn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fcn-m'    
        model = Model(PARAMS, task="segmentation")
    elif(model_choice == "resnet"):
        PARAMS['architect_settings']['backbone']['name'] = 'resnet-s'    
        model = Model(PARAMS, task="classification")
    elif(model_choice == "fasterrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'fasterrcnn-s'    
        model = Model(PARAMS, task="detection")
    elif(model_choice == "maskrcnn"):
        PARAMS['architect_settings']['backbone']['name'] = 'maskrcnn-s'    
        model = Model(PARAMS, task="detection")
    else:
        model = Model.load_from_checkpoint(f"models/{model_choice}.ckpt").cpu()
    model.eval()
    transforms = model.model.preprocess
    tensor_image = transforms(image)
    with torch.no_grad():
        y_hat = model(tensor_image.unsqueeze(0))
        if(model.task == "classification"):
            preds = torch.softmax(y_hat, dim=-1).tolist()
            labels = {class_names[k]: float(v) for k, v in enumerate(preds[0][:-1])}
        elif(model.task == "segmentation"):
            num_classes = y_hat.shape[1]
            masks = y_hat[0]
            classes_masks = masks.argmax(0) == torch.arange(num_classes)[:, None, None]
            tensor_image = inv_normalize(tensor_image)
            detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8), 
                                              masks=classes_masks, alpha=.6)
            detection = detection.numpy().transpose(1, 2, 0) / 255.
        elif(model.task == "detection"):
            if("maskrcnn" in model_choice):
                boolean_masks = [out['masks'][out['scores'] > .75] > 0.5
                                for out in y_hat][0]
                detection = draw_segmentation_masks((tensor_image * 255.).to(torch.uint8),
                                                    boolean_masks.squeeze(1), alpha=0.8)
                detection = detection.numpy().transpose(1, 2, 0) / 255.
            else:
                detection = draw_bounding_boxes((tensor_image * 255.).to(torch.uint8), 
                                                    boxes=y_hat[0]["boxes"][:5],
                                                    colors="red",
                                                    width=5)
                detection = detection.numpy().transpose(1, 2, 0) / 255.

    return labels, detection

Couldn't find program: 'false'


In [7]:
%%script false --no-raise-error

import gradio as gr

title = "Application Demo "
description = "# A Demo of Wrapping Pretrained Networks"
example_list = [["examples/" + example] for example in os.listdir("examples")]


with gr.Blocks() as demo:
    demo.title = title
    gr.Markdown(description)
    with gr.Row():
        with gr.Column():
            model = gr.Dropdown(model_list, label="Select Model", interactive=True)
            im = gr.Image(type="pil", label="input image")
            label_conv = gr.Label(label="Predictions", num_top_classes=4)
        with gr.Column():
            im_detection = gr.Image(type="pil", label="Detection")
            btn = gr.Button(value="predict")
    btn.click(predict, inputs=[im, model], outputs=[label_conv, im_detection])
    gr.Examples(examples=example_list, inputs=[im, model], outputs=[label_conv, im_detection])
      

demo.launch(share=True)

Couldn't find program: 'false'
